# "O" for the Open/Closed Principle

In our pizza restaurant, Pythonia, the waiters and waitresses are very highly skilled at their job. They obey the **single responsibility principle** by doing only one sort of thing (serving customers).

However, they only speak English. Let’s imagine we wanted to train them to communicate in Italian for the convenience of all the Italian-speaking customers.

This shouldn’t **modify** their existing skillset - they will still carry the pizzas in the same way and greet English-speaking customers in the same way. But after their Italian training, they have **extended** their skillset.

In this way, you could say that the waiters and waitresses are **open for extension** but **closed for modification**.

<blockquote style="padding: 1em; border-radius: 5px; font-style: italic;">
Could the same idea be applied to classes in Python?
</blockquote>

Great guess! This brings us to the **open/closed principle**, which states that:

<blockquote style="padding: 1em; border-radius: 5px; font-style: italic;">
A class should be open to extension but closed to modification.
</blockquote>

## Why Use the Open/Closed Principle?

If you **aren’t modifying the existing code**, you know **you aren’t breaking any of it**. All the new functionality is contained in the newly added function(s) or class(es).

The hardest part about this principle is recognizing _when_ it can be applied _before_ you begin coding. Here are a couple of **guidelines** to help you recognize when open/closed may be applicable.

- When you have **algorithms** that perform a calculation (cost, tax, game score, etc.): the algorithm will likely change over time.
- When you have **data coming or going from the system**: the endpoint (file, database, another system) is likely to change. So is the actual format of the data.

## How Do We Apply the Open/Closed Principle to Our Code?

One place for modification is in the winner evaluation. In the last chapter, we created an `evaluate_game` function in the GameController class. What if we changed the game so that the lowest card wins? Then we’d have to modify GameController - maybe by adding a Boolean parameter to say "find the high card winner" vs. "find the low card winner":


In [ ]:
class GameController:
    def __init__(self, deck, view, high_card_wins=True):
        # Model
        self.players = []
        self.deck = deck

        # View
        self.view = view

        # Controller
        self.high_card_wins = high_card_wins

    def evaluate_game(self):
        best_rank = None
        best_rank_suit = None
        best_candidate = None

        for player in self.players:
            this_rank = player.hand.card_by_index(0).rank
            this_suit = player.hand.card_by_index(0).suit
            if (
                best_rank is None
                or (
                    self.high_card_wins
                    and (
                        (this_rank > best_rank)
                        or (this_rank == best_rank and this_suit > best_rank_suit)
                    )
                )
                or (
                    not self.high_card_wins
                    and (
                        (this_rank < best_rank)
                        or (this_rank == best_rank and this_suit < best_rank_suit)
                    )
                )
            ):
                best_candidate = player.name
                best_rank = this_rank
                best_rank_suit = this_suit

        return best_candidate

Ugh, that’s complicated enough! But what happens if we wanted to add even more rules?! 😨 This class becomes difficult to understand, test, and maintain.

And furthermore, now everyone else who is using our `GameController` class has to implement this Boolean parameter. Not a good idea.

<blockquote style="padding: 1em; border-radius: 5px; font-style: italic;">
So, what should we do?
</blockquote>

Really, the winner evaluation method should be defined outside of the GameController class, in a separate function or class.

We can make a separate GameEvaluator class to do the evaluation and pass an **instance** of it into GameController. Then `evaluate_game` calls the relevant method from this GameEvaluator object.

Then when we add alternative rules, such as the low card winning, we can create a `LowCardGameEvaluator` and use that instead.

Let’s try this together!


So how well are we following SOLID principles so far?

- GameController still has a **single responsibility**.
- GameEvaluator is a new class, also with a very clear single responsibility.
- GameController remains open to an extension of the winner evaluation rules but closed to modification. Alternative rules for evaluating the winner can be added without affecting existing implementations.

<blockquote style="padding: 1em; border-radius: 5px; font-style: italic;">
But wait - it’s possible to play a game with more complicated rules, where the winner somehow depends on something else like the cards still in the deck. We would still have to modify the GameController class!
</blockquote>

Yes, you’re right. It’s unrealistic to write code that will never need to be modified, and sometimes **breaking changes** are indeed required, which cause other code to be modified to accommodate the changes.

But the closer you are following the open/closed principle, the less frequent and less troublesome these changes will be.

Let’s Recap!
- The open/closed principle says that classes should be open to extension but closed to modification.
- In existing systems, it might take some rework to get the code in a position to take advantage of open/closed.

-------------------------------------------------------------------------------

## Principio Abierto/Cerrado - OCP

Acuñado en 1988 por Bertrand Meyer:

<blockquote style="padding: 1em; border-radius: 5px; font-style: italic;">
Las entidades de software (funciones, clases, módulos, etc.) debe ser abiertas a la extensión, pero cerradas a la modificación.
</blockquote>

El objetivo es hacer que el sistema sea fácil de extender sin incurrir en un alto impacto de cambio. Los arquitectos de software logran este objetivo separando la funcionalidad del sistema en componentes prestando atención en cómo, por qué, y cuándo dicha funcionalidad cambia. Luego, organizan esos componentes en una jerarquía de dependencia que protege los componentes de nivel superior de los cambios realizados en los componentes de nivel inferior (Martin, 2017, cap.8).

### La abstracción es la clave

¿Cómo lograr modificar el comportamiento de un módulo sin cambiar el código fuente de ese módulo? En programación orientada a objetos es posible crear abstracciones que son fijas y sin embargo representan un grupo ilimitado de comportamientos posibles. Las abstracciones son clases _base_ abstractas, y el grupo ilimitado de
comportamientos posibles está representado por todas las clases derivadas posibles.

### Conclusión

Una solución que permite poner en práctica este principio implica abstraer nuestros objetos en clases _base_. Los patrones de diseño _Strategy_ y _Template Method_ son los más comunes para satisfacer el OCP.


### Ejemplo

- Tarea 1: Diseña un descuento para lanzar la aplicación (15%).
- Tarea 2: Diseña el manager de descuentos para gestionar el descuento.
- Tarea 3: Diseña un descuento para jugadores gold (30).
- Tarea 4: Diseña un descuento para el Black Friday (35).
- Tarea 5: Diseña un descuento para el Navidad (50).


In [27]:
from dataclasses import dataclass
from gzip import READ

# app/module/business_rules/discount.py
@dataclass
class DiscountManager:
    price: float

    def apply(self, discount: int):
        return self.price * discount / 100


# app/module/business_rules/release_discount.py
@dataclass
class ReleaseDiscount(DiscountManager):
    def apply(self, discount: int = 15):
        return super().apply(discount)


# app/module/business_rules/gold_player_discount.py
@dataclass
class GoldPlayerDiscount(DiscountManager):
    def apply(self, discount: int = 30):
        return super().apply(discount)


# app/module/business_rules/black_friday_discount.py
@dataclass
class BlackFridayDiscount(DiscountManager):
    def apply(self, discount: int = 35):
        return super().apply(discount)


# app/module/business_rules/black_friday_discount.py
@dataclass
class ChristmasDiscount(DiscountManager):
    def apply(self, discount: int = 50):
        return super().apply(discount)


a = ReleaseDiscount(price=1000)
b = GoldPlayerDiscount(price=1000)
c = BlackFridayDiscount(price=1000)
d = ChristmasDiscount(price=1000)
print(a.apply(), b.apply(), c.apply(), d.apply())

150.0 300.0 350.0 500.0
